### Import modules

For more information on the schiene package, please see here: https://github.com/kennell/schiene/blob/master/README.md

And thanks to the developers fot this great and very helpful package.

In [1]:
from pymongo import MongoClient
import schiene
import datetime

### Connect to local MongoDB instance
I have created to collections. One for the trains from home to destination and one for the other direction

In [2]:
client = MongoClient("localhost", 27017)
db = client["sbahnmuc"]
tocoll = "to"
db_tocoll = db[tocoll]

fromcoll = "from"
db_fromcoll = db[fromcoll]

In [ ]:
#create index on day - only after creating the collections
#db_tocoll.create_index("date")
#db_fromcoll.create_index("date")

### Define home and destination

In [8]:
home = 'Puchheim Bahnhof Nordseite'
dest = 'München Trudering'

### Initiate schiene object and get current connections towards dest and towards home

In [36]:
s = schiene.Schiene()
toconnlist = s.connections(home, dest)
fromconnlist = s.connections(dest, home)

These 2 lists contain dictionaries per current connection. Only the ones whcih contain a subdict named "delay" is interesting here. So in the loops below only the interesting ones are filtered out. Also some more information is added or deleted.

In [37]:
toconnlist

[{'details': 'http://mobile.bahn.de/bin/mobil/query.exe/dox?ld=42243&n=1&i=li.027737243.1575895896&rt=1&use_realtime_filter=1&co=C0-0&vca&HWAI=CONNECTION$C0-0!details=opened!detailsVerbund=opened!&&newVerbundtarif=yes&verbund=MVV&outConId=C0-0&tplVarsDBStyle=yes&',
  'departure': '13:18',
  'arrival': '14:00',
  'transfers': 1,
  'time': '0:42',
  'products': ['S', 'U'],
  'price': None,
  'ontime': True,
  'canceled': False},
 {'details': 'http://mobile.bahn.de/bin/mobil/query.exe/dox?ld=42243&n=1&i=li.027737243.1575895896&rt=1&use_realtime_filter=1&co=C0-1&vca&HWAI=CONNECTION$C0-1!details=opened!detailsVerbund=opened!&&newVerbundtarif=yes&verbund=MVV&outConId=C0-1&tplVarsDBStyle=yes&',
  'departure': '13:18',
  'arrival': '14:05',
  'transfers': 1,
  'time': '0:47',
  'products': ['S', 'U'],
  'price': None,
  'ontime': True,
  'canceled': False},
 {'details': 'http://mobile.bahn.de/bin/mobil/query.exe/dox?ld=42243&n=1&i=li.027737243.1575895896&rt=1&use_realtime_filter=1&co=C0-2&vca&

In [12]:
for conn in toconnlist:
    if"delay" in conn.keys():
        
        del conn["details"]
        del conn["transfers"]
        del conn["price"]
        conn["date"] = str(datetime.date.today())
        conn["_id"] = str(conn["date"]) + "_" + conn["departure"]
        conn["timestamp"] = str(datetime.datetime.now())
        conn["total_delay"] = conn["delay"]["delay_departure"] + conn["delay"]["delay_arrival"]
                
        db_tocoll.remove({"_id": conn["_id"]})
        
        db_tocoll.insert_one(conn)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  if sys.path[0] == '':


In [13]:
for conn in fromconnlist:
    if"delay" in conn.keys():
        
        del conn["details"]
        del conn["transfers"]
        del conn["price"]
        conn["date"] = str(datetime.date.today())
        conn["_id"] = str(conn["date"]) + "_" + conn["departure"]
        conn["timestamp"] = str(datetime.datetime.now())
        conn["total_delay"] = conn["delay"]["delay_departure"] + conn["delay"]["delay_arrival"]
               
        db_fromcoll.remove({"_id": conn["_id"]})
        
        db_fromcoll.insert_one(conn)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  if sys.path[0] == '':


### Check out the results in MongoDB
It seems that the first to train has 3 minutes delay.
All others are currently fine.

In [40]:
r = db_tocoll.find()
for l in r:
    print(l)

{'_id': '2019-12-09_13:18', 'departure': '13:18', 'arrival': '14:04', 'time': '0:43', 'products': ['S'], 'ontime': False, 'delay': {'delay_departure': 3, 'delay_arrival': 0}, 'canceled': False, 'date': '2019-12-09', 'timestamp': '2019-12-09 13:21:27.668641', 'total_delay': 3}
{'_id': '2019-12-09_13:38', 'departure': '13:38', 'arrival': '14:14', 'time': '0:36', 'products': ['S'], 'ontime': True, 'delay': {'delay_departure': 0, 'delay_arrival': 0}, 'canceled': False, 'date': '2019-12-09', 'timestamp': '2019-12-09 13:21:27.669638', 'total_delay': 0}


In [41]:
r2 = db_fromcoll.find()
for l in r2:
    print(l)

{'_id': '2019-12-09_13:43', 'departure': '13:43', 'arrival': '14:21', 'time': '0:38', 'products': ['S'], 'ontime': True, 'delay': {'delay_departure': 0, 'delay_arrival': 0}, 'canceled': False, 'date': '2019-12-09', 'timestamp': '2019-12-09 13:21:30.515328', 'total_delay': 0}
